### csv

In [ ]:
import os
import openai
import pandas as pd


def csv_to_txt(csv_path):
    df = pd.read_csv(csv_path)
    df.to_csv("./openai_gen_dataset.txt", encoding='utf-8', sep=',', index=False)

def txt_to_csv(txt_path, save_csv_path):
    df = pd.read_csv(txt_path, on_bad_lines='skip')
    df.to_csv(save_csv_path, encoding='utf-8-sig', index=False)
    print(f"{save_csv_path}에 파일이 저장되었습니다!")

def make_dataset():
    completion = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role":"system", "content": "당신은 사용자가 입력하는 예시 데이터와 유사하게 데이터를 새롭게 생성하는 챗봇입니다.\n데이터는 현실 상황을 반영해서 다양하고 중복되지 않게 데이터를 10개 생성해주세요."},
            {"role":"user", "content": """<<예시 데이터>>
HOST_NAME,CLASS_NAME,OBJECT_NAME,EVENT_NAME,BAD_LEVEL,SENSOR_MSG,SMS_MSG,SENSOR_BBT_FLAG,BLACKOUT_FLAG,BLACKOUT_START,BLACKOUT_END,SENSOR_OWNER,IMPUTE_DEPT,IMPUTE_PART,SOP_URL,DESCRIPTION,CHANGE_REASON,ALAM_ACTION,PROCESS_NAME,SENSOR_TYPE,COMMON_YN,SENSOR_VALUE,REMARK
P1BEESFA1,PROCESS_HANG,FDC_P1B_CLN_5,CHECK_MES_PROCESS,A,PT_EES_FDC-:PROCESS_DOWN,MES 프로세스가 다운되었습니다.,Y,N,NULL,NULL,황지환,EES,FDC,.,MES 프로세스가 다운되었습니다.,NULL,PROCESS_START,FDC_CLN,CHECK_PROCESS,Y,"{'process_status': False, 'process_name': 'MES_MAIN', 'uptime': '0min'}",원인: MES 프로세스가 예기치 않게 종료됨 (Spec: 프로세스 다운 시 즉각 재시작 필요). 조치: 프로세스 재시작 및 로그 분석.
P2CEMSFA2,NETWORK_ERROR,EMS_P2C_NET_3,CHECK_NETWORK_STATUS,B,PT_CEM_EMS-,네트워크에 연결할 수 없습니다.,N,Y,2023-10-01 8:00,2023-10-01 10:00,김민수,CEM,EMS,http://sop.cem.net/network_error,네트워크 연결이 끊겼습니다.,서버 유지보수,RESTART_NETWORK,EMS_NET,CHECK_NETWORK,N,"{'network_status': False, 'ping_response': 'timeout'}",현상: 네트워크 연결 불가. 원인: 서버 유지보수 작업으로 인한 계획된 중단 (Spec: 네트워크 패킷 손실이 20% 이상일 경우 오류로 간주). 조치: 유지보수 완료 후 네트워크 재연결.
NODE_X1001,POWER_SURGE,PWR_X1001_MAIN,CHECK_POWER_SURGE,D,PT_X1001_PWR,전력 서지가 감지되었습니다.,N,Y,2023-10-06 12:30,2023-10-06 12:45,김진수,AES,PWR,http://sop.aes.net/power_surge,전력 서지로 인해 시스템 보호가 필요합니다.,외부 전력 문제,SHUTDOWN_SYSTEM,MAIN_PWR,CHECK_POWER,N,"{'voltage': 245, 'normal_range': '220-240V', 'duration': '15min'}",원인: 전력 서지 발생 (245V) (Spec: 전력 소비가 2kW를 초과하면 과도한 소비로 간주). 조치: 전원 안정화 장치 가동.
SVR_B12,FAN_FAILURE,FAN_B12_SYS,CHECK_FAN_STATUS,C,PT_B12_FAN,팬이 작동하지 않습니다.,Y,N,NULL,NULL,윤영호,BBS,FAN,http://sop.bbs.net/fan_failure,서버 냉각 팬이 작동을 멈췄습니다.,부품 결함,REPLACE_FAN,SYS_FAN,CHECK_HARDWARE,Y,"{'fan_speed': 1200, 'normal_range': '2000-3000rpm', 'temperature': '45C'}",원인: 팬 베어링 마모로 인한 팬 속도 저하 (Spec: 팬 속도가 정상 범위(2000-3000rpm)를 벗어날 경우 교체 필요). 조치: 팬 교체.

- HOST_NAME과 다양한 EVENT로 데이터를 생성합니다.
- REMARK에는 "원인"과 "조치"는 항상 포함하고, 5개의 데이터는 "대책"과 "현상"을 포함하고, 나머지 5개는 포함하지 않습니다.
- REMARK의 "원인"에는 Spec을 항상 포함시킵니다. Spec은 SENSOR_VALUE에 따라 알아서 기준점을 설정해서 생성합니다.
- 생성한 데이터에는 컬럼명이 포함되지 않아야 합니다. 데이터만 생성하고 데이터는 바로 다음 줄에 생성하세요."""}
        ]   
    )
    
    data = completion.choices[0].message.content
    if "\n\n" in data:
        data = data.replace("\n\n", "\n")
    return data




if __name__=="__main__":
    openai.api_key = ""
    
    CSV_PATH = "/home/ubuntu/EES_sample_dataset.csv"
    TXT_PATH = "./openai_gen_dataset.txt"
    NEW_CSV_PATH = "./EES_openai_dataset.csv"


    if os.path.isfile(TXT_PATH):
        print("txt 파일 이미 있음! PASS~!!!")
    else:
        csv_to_txt(CSV_PATH)
    
    
    txt_file = open(TXT_PATH, "a", encoding="utf-8")

    
    # for i in range(100):
    #     print("생성중...")
    #     data = make_dataset()
    #     print("생성된 데이터: ")
    #     print(data, '\n\n')
    #     print("==========================================================================================")
    #     txt_file.write(data)
    #     txt_file.write('\n')
    # txt_file.close()
        
    txt_to_csv(TXT_PATH, NEW_CSV_PATH)    

### json

In [ ]:
import os
import json
import openai

def json_to_txt(json_path):
    with open(json_path, 'r', encoding='utf-8') as json_file:
        data = json.load(json_file)
    with open("./openai_gen_dataset.txt", 'w', encoding='utf-8') as txt_file:
        json.dump(data, txt_file, ensure_ascii=False, indent=2)

def txt_to_json(txt_path, save_json_path):
    with open(txt_path, 'r', encoding='utf-8') as txt_file:
        data = json.load(txt_file)
    with open(save_json_path, 'w', encoding='utf-8') as json_file:
        json.dump(data, json_file, ensure_ascii=False, indent=2)
    print(f"{save_json_path}에 파일이 저장되었습니다!")

def make_dataset(original_data):
    completion = openai.chat.completions.create(
        model="gpt-4-1106-preview",
        messages=[
            {"role": "system", "content": "당신은 사용자가 입력하는 예시 데이터와 유사하게 데이터를 새롭게 생성하는 챗봇입니다. 데이터는 현실 상황을 반영해서 다양하고 중복되지 않게 데이터를 10개 생성해주세요."},
            {"role": "user", "content": f"""<<예시 데이터>>
{json.dumps(original_data, ensure_ascii=False, indent=2)}

- 위 예시 데이터와 유사한 구조로 새로운 데이터를 5개 생성해주세요.
- 생성된 데이터는 JSON 형식이어야 합니다.
- 데이터의 키와 구조는 유지하되, 값은 다양하게 변경해주세요.
- REMARK에는 "원인"과 "조치"는 항상 포함하고, 5개의 데이터는 "대책"과 "현상"을 포함하고, 나머지 5개는 포함하지 않습니다.
- REMARK의 "원인"에는 Spec을 항상 포함시킵니다. Spec은 SENSOR_VALUE에 따라 알아서 기준점을 설정해서 생성합니다."""}
        ]   
    )
    
    return json.loads(completion.choices[0].message.content)

if __name__ == "__main__":
    openai.api_key = ""
    
    JSON_PATH = "/home/ubuntu/EES_sample_dataset.json"
    TXT_PATH = "./openai_gen_dataset.txt"
    NEW_JSON_PATH = "./EES_openai_dataset.json"

    if os.path.isfile(TXT_PATH):
        print("txt 파일 이미 있음! PASS~!!!")
    else:
        json_to_txt(JSON_PATH)
    
    with open(JSON_PATH, 'r', encoding='utf-8') as json_file:
        original_data = json.load(json_file)
    
    with open(TXT_PATH, "r+", encoding="utf-8") as txt_file:
        existing_data = json.load(txt_file)
        
        # for i in range(100):  # 100개의 새로운 데이터 생성
        #     print(f"{i+1}번째 데이터 생성중...")
        #     new_data = make_dataset(original_data)
        #     existing_data.extend(new_data)
        #     print("생성 완료")
        
        txt_file.seek(0)
        json.dump(existing_data, txt_file, ensure_ascii=False, indent=2)
        txt_file.truncate()
    
    txt_to_json(TXT_PATH, NEW_JSON_PATH)

## Test

In [ ]:
import json
import random
from openai import OpenAI

# OpenAI 클라이언트 초기화
client = OpenAI(api_key="")

# JSON 파일 경로 정의
json_paths = [
    # '/home/user/hjkim/contextual/claim/merged_parsed_patent_data.json'
    '/home/user/hjkim/contextual/law/law_json/all_law.json'
    # '/home/user/hjkim/contextual/judgement/judgement.json'
]

# JSON 파일 읽기 함수
def read_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)

# 모든 JSON 파일에서 데이터 읽기
documents = []
for path in json_paths:
    try:
        data = read_json(path)
        if isinstance(data, list):
            documents.extend(data)
        else:
            documents.append(data)
    except Exception as e:
        print(f'Error reading {path}: {e}')

# GPT를 사용하여 데이터셋 생성 함수
def make_dataset(docs):
    prompt = """당신은 변리사가 업무 중 할 법한 청구항, 요약, 법, 판례를 활용한 질문들을 생성하는 챗봇입니다.
    특정 청구항이나 특정 판례와 관련된 내용이라면 documents에 특허번호: (특허번호) |, 판례 일련번호: (판례 일련번호) | 를 추가 후 그 뒤로 내용을 작성해주세요.
    다음 형식으로 10개의 다양하고 중복되지 않는 데이터를 생성해주세요:
    [
        {
            "input": "(생성한 변리사가 업무 중 할 법한 질문)",
            "documents": "(첨부한 경로들(특허 전문, 법, 청구항, 요약, 판례)를 활용하여 input에 대한 대답의 근거 내용)",
            "output": "(생성한 input에 대한 생성한 대답)"
        },
        ...
    ]
    """
    
    sample_docs = random.sample(docs, min(10, len(docs)))
    prompt += "\n\n참고할 수 있는 문서 샘플:\n" + json.dumps(sample_docs, ensure_ascii=False)

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "system", "content": prompt}]
    )

    return json.loads(response.choices[0].message.content)

# 메인 실행 부분
if __name__ == "__main__":
    print("데이터셋 생성 시작...")
    new_data = make_dataset(documents)
    
    print("\n생성된 데이터:")
    for i, item in enumerate(new_data, 1):
        print(f"\n{i}번째 데이터:")
        print(json.dumps(item, ensure_ascii=False, indent=2))
    
    print(f"\n총 {len(new_data)}개의 데이터가 생성되었습니다.")

데이터셋 생성 시작...


RateLimitError: Error code: 429 - {'error': {'message': 'Request too large for gpt-4o in organization org-Op6u5lUJ3xK5y7Sm02aL1EIY on tokens per min (TPM): Limit 800000, Requested 53252488. The input or output tokens must be reduced in order to run successfully. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}

In [ ]:
# # 메인 실행 부분
# if __name__ == "__main__":
#     existing_data = []
    
#     for i in range(100):  # 100번 반복하여 총 1000개 생성
#         print(f"{i+1}번째 데이터셋 생성중...")
#         new_data = make_dataset(documents)
#         existing_data.extend(new_data)
#         print("생성 완료")

#     # 결과를 JSON 파일로 저장
#     output_path = '/path/to/your/output/new_dataset.json'
#     with open(output_path, 'w', encoding='utf-8') as f:
#         json.dump(existing_data, f, ensure_ascii=False, indent=2)

#     print(f"총 {len(existing_data)}개의 데이터가 {output_path}에 저장되었습니다.")

## 파일 내 키 나눠서 진행

In [ ]:
# 법
import json
from openai import OpenAI
import os
import re

def make_dataset(prompt):
    client = OpenAI(api_key="")
    
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role":"system", "content": "당신은 변리사가 업무 중 할 법한 법을 활용한 질문들을 생성하는 챗봇입니다. \n데이터는 변리사의 업무상황을 반영해서 다양하고 중복되지 않게 데이터를 10개 생성해주세요. 반드시 유효한 JSON 형식으로 응답해주세요."},
            {"role":"user", "content": prompt}
        ]   
    )
    
    data = completion.choices[0].message.content
    return data

def process_law_json(input_file, output_file):
    try:
        with open(input_file, 'r', encoding='utf-8') as f:
            input_data = json.load(f)
    except json.JSONDecodeError as e:
        print(f"JSON 파싱 오류: {e}")
        return
    except IOError as e:
        print(f"파일 읽기 오류: {e}")
        return

    if not isinstance(input_data, dict):
        print("입력 데이터가 딕셔너리 형식이 아닙니다.")
        return

    output_data = {}

    for law_name, law_content in input_data.items():
        print(f"법령 '{law_name}' 처리 중...")
        
        law_summary = law_content[:1000] if len(law_content) > 1000 else law_content

        prompt = f"""다음 법령에 대해 변리사가 업무 중 할 법한 질문과 답변을 생성해주세요:

법령명: {law_name}
법령 내용 요약:
{law_summary}

<<예시 데이터 형식>>
[
    {{
        "input": "(생성한 변리사가 업무 중 업무지원 챗봇에게 할 법한 법과 관련된 질문)",
        "documents": "(input에 대한 대답의 근거 내용)",
        "output": "(생성한 input에 대한 생성한 대답)"
    }}
]

- 생성한 데이터는 반드시 유효한 JSON 형식으로 생성하세요."""

        law_data = []
        for i in range(100):  # 100번 반복
            print(f"  {i+1}/100 세트 생성 중...")
            raw_data = make_dataset(prompt)
            
            # 전처리: 불필요한 문자 제거
            raw_data = re.sub(r'```json\s*|\s*```', '', raw_data)
            
            try:
                json_data = json.loads(raw_data)
                law_data.extend(json_data)  # 10개씩 생성된 데이터를 law_data에 추가
            except json.JSONDecodeError as e:
                print(f"  세트 {i+1}에 대한 데이터를 JSON으로 파싱할 수 없습니다. 오류: {e}")
                print("  원본 데이터:", raw_data)
                # 파싱 실패 시 원본 데이터 저장
                law_data.append({"raw_data": raw_data})

        output_data[law_name] = law_data
        print(f"법령 '{law_name}' 처리 완료 (총 {len(law_data)}개 데이터 생성)")

    try:
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(output_data, f, ensure_ascii=False, indent=2)
        print(f"처리 완료. 결과가 {output_file}에 저장되었습니다.")
    except IOError as e:
        print(f"결과 저장 중 오류 발생: {e}")

# 메인 실행 부분
if __name__ == "__main__":
    input_file = '/media/user/datadisk/hjkim/contextual/law/law_json/all_law.json'
    output_file = 'generated_law_qa.json'
    
    if not os.path.exists(input_file):
        print(f"입력 파일이 존재하지 않습니다: {input_file}")
    else:
        process_law_json(input_file, output_file)

법령 'civil_law' 처리 중...
  1/100 세트 생성 중...
  2/100 세트 생성 중...
  3/100 세트 생성 중...
  4/100 세트 생성 중...
  5/100 세트 생성 중...
  6/100 세트 생성 중...
  7/100 세트 생성 중...
  8/100 세트 생성 중...
  9/100 세트 생성 중...
  10/100 세트 생성 중...
  11/100 세트 생성 중...
  12/100 세트 생성 중...
  13/100 세트 생성 중...
  14/100 세트 생성 중...
  15/100 세트 생성 중...
  16/100 세트 생성 중...
  17/100 세트 생성 중...
  18/100 세트 생성 중...
  19/100 세트 생성 중...
  20/100 세트 생성 중...
  21/100 세트 생성 중...
  22/100 세트 생성 중...
  23/100 세트 생성 중...
  24/100 세트 생성 중...
  25/100 세트 생성 중...
  26/100 세트 생성 중...
  27/100 세트 생성 중...
  28/100 세트 생성 중...
  29/100 세트 생성 중...
  30/100 세트 생성 중...
  31/100 세트 생성 중...
  32/100 세트 생성 중...
  33/100 세트 생성 중...
  34/100 세트 생성 중...
  35/100 세트 생성 중...
  36/100 세트 생성 중...
  37/100 세트 생성 중...
  38/100 세트 생성 중...
  39/100 세트 생성 중...
  40/100 세트 생성 중...
  41/100 세트 생성 중...
  42/100 세트 생성 중...
  43/100 세트 생성 중...
  44/100 세트 생성 중...
  45/100 세트 생성 중...
  46/100 세트 생성 중...
  47/100 세트 생성 중...
  48/100 세트 생성 중...
  49/100 세트 생성 중...
  50/1

In [ ]:
# 법 test 1개씩만
import json
from openai import OpenAI
import os
import re

def make_dataset(prompt):
    client = OpenAI(api_key="")
    
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role":"system", "content": "당신은 변리사가 업무 중 할 법한 법을 활용한 질문들을 생성하는 챗봇입니다. \n데이터는 변리사의 업무상황을 반영해서 다양하고 중복되지 않게 데이터를 5개 생성해주세요. 반드시 유효한 JSON 형식으로 응답해주세요."},
            {"role":"user", "content": prompt}
        ]   
    )
    
    data = completion.choices[0].message.content
    return data

def process_law_json(input_file, output_file):
    try:
        with open(input_file, 'r', encoding='utf-8') as f:
            input_data = json.load(f)
    except json.JSONDecodeError as e:
        print(f"JSON 파싱 오류: {e}")
        return
    except IOError as e:
        print(f"파일 읽기 오류: {e}")
        return

    if not isinstance(input_data, dict):
        print("입력 데이터가 딕셔너리 형식이 아닙니다.")
        return

    output_data = {}

    for law_name, law_content in input_data.items():
        print(f"법령 '{law_name}' 처리 중...")
        
        law_summary = law_content[:1000] if len(law_content) > 1000 else law_content

        prompt = f"""다음 법령에 대해 변리사가 업무 중 할 법한 질문과 답변을 생성해주세요:

법령명: {law_name}
법령 내용 요약:
{law_summary}

<<예시 데이터 형식>>
[
    {{
        "input": "(생성한 변리사가 업무 중 업무지원 챗봇에게 할 법한 법과 관련된 질문)",
        "documents": "(input에 대한 대답의 근거 내용)",
        "output": "(생성한 input에 대한 생성한 대답)"
    }}
]

- 생성한 데이터는 반드시 유효한 JSON 형식으로 생성하세요."""

        raw_data = make_dataset(prompt)
        
        # 전처리: 불필요한 문자 제거
        raw_data = re.sub(r'```json\s*|\s*```', '', raw_data)
        
        try:
            json_data = json.loads(raw_data)
            output_data[law_name] = json_data
        except json.JSONDecodeError as e:
            print(f"  데이터를 JSON으로 파싱할 수 없습니다. 오류: {e}")
            print("  원본 데이터:", raw_data)
            # 파싱 실패 시 원본 데이터 저장
            output_data[law_name] = [{"raw_data": raw_data}]

        print(f"법령 '{law_name}' 처리 완료")

    try:
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(output_data, f, ensure_ascii=False, indent=2)
        print(f"처리 완료. 결과가 {output_file}에 저장되었습니다.")
    except IOError as e:
        print(f"결과 저장 중 오류 발생: {e}")

# 메인 실행 부분
if __name__ == "__main__":
    input_file = '/media/user/datadisk/hjkim/contextual/law/law_json/all_law.json'
    output_file = 'generated_law_qa_test5.json'
    
    if not os.path.exists(input_file):
        print(f"입력 파일이 존재하지 않습니다: {input_file}")
    else:
        process_law_json(input_file, output_file)

법령 'civil_law' 처리 중...
법령 'civil_law' 처리 완료
법령 'design_protection_law' 처리 중...
법령 'design_protection_law' 처리 완료
법령 'patent_law' 처리 중...
법령 'patent_law' 처리 완료
법령 'trademark_law' 처리 중...
법령 'trademark_law' 처리 완료
법령 'utility_model_law' 처리 중...
법령 'utility_model_law' 처리 완료
법령 'civil_procedure_law' 처리 중...
법령 'civil_procedure_law' 처리 완료
처리 완료. 결과가 generated_law_qa_test5.json에 저장되었습니다.


In [ ]:
# 청구항
import json
from openai import OpenAI
import os
import re

def make_dataset(prompt):
    client = OpenAI(api_key="")
    
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role":"system", "content": "당신은 변리사가 업무 중 할 법한 특허 청구항과 요약을 활용한 질문들을 생성하는 챗봇입니다. \n데이터는 변리사의 업무상황을 반영해서 다양하고 중복되지 않게 데이터를 5개 생성해주세요. 반드시 유효한 JSON 형식으로 응답해주세요."},
            {"role":"user", "content": prompt}
        ]   
    )
    
    data = completion.choices[0].message.content
    return data

def process_patent_json(input_file, output_file):
    try:
        with open(input_file, 'r', encoding='utf-8') as f:
            input_data = json.load(f)
    except json.JSONDecodeError as e:
        print(f"JSON 파싱 오류: {e}")
        return
    except IOError as e:
        print(f"파일 읽기 오류: {e}")
        return

    if not isinstance(input_data, dict):
        print("입력 데이터가 딕셔너리 형식이 아닙니다.")
        return

    output_data = {}

    for patent_number, patent_content in input_data.items():
        print(f"특허 번호 '{patent_number}' 처리 중...")
        
        claim1 = patent_content.get("청구항1", "")
        other_claims = patent_content.get("그 외 청구항", "")
        summary = patent_content.get("요약", "")

        prompt = f"""다음 특허 청구항과 요약에 대해 변리사가 업무 중 할 법한 질문과 답변을 생성해주세요:

특허 번호: {patent_number}
청구항1: {claim1[:500]}...
그 외 청구항: {other_claims[:500]}...
요약: {summary[:500]}...

<<예시 데이터 형식>>
[
    {{
        "input": "(생성한 변리사가 업무 중 업무지원 챗봇에게 할 법한 특허 청구항과 요약에 관련된 질문)",
        "documents": "특허번호: {patent_number} | (input에 대한 대답의 근거 내용)",
        "output": "(생성한 input에 대한 생성한 대답)"
    }}
]

- 생성한 데이터는 반드시 유효한 JSON 형식으로 생성하세요."""

        print("  데이터 생성 중...")
        raw_data = make_dataset(prompt)
        
        # 전처리: 불필요한 문자 제거
        raw_data = re.sub(r'```json\s*|\s*```', '', raw_data)
        
        try:
            json_data = json.loads(raw_data)
            output_data[patent_number] = json_data
        except json.JSONDecodeError as e:
            print(f"  데이터를 JSON으로 파싱할 수 없습니다. 오류: {e}")
            print("  원본 데이터:", raw_data)
            # 파싱 실패 시 원본 데이터 저장
            output_data[patent_number] = [{"raw_data": raw_data}]

        print(f"특허 번호 '{patent_number}' 처리 완료 (총 {len(json_data)}개 데이터 생성)")

    try:
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(output_data, f, ensure_ascii=False, indent=2)
        print(f"처리 완료. 결과가 {output_file}에 저장되었습니다.")
    except IOError as e:
        print(f"결과 저장 중 오류 발생: {e}")

# 메인 실행 부분
if __name__ == "__main__":
    input_file = '/media/user/datadisk/hjkim/contextual/claim/merged_parsed_patent_data.json'
    output_file = 'generated_patent_qa.json'
    
    if not os.path.exists(input_file):
        print(f"입력 파일이 존재하지 않습니다: {input_file}")
    else:
        process_patent_json(input_file, output_file)

특허 번호 '1020170035102' 처리 중...
  데이터 생성 중...
특허 번호 '1020170035102' 처리 완료 (총 5개 데이터 생성)
특허 번호 '1020170035108' 처리 중...
  데이터 생성 중...
특허 번호 '1020170035108' 처리 완료 (총 5개 데이터 생성)
특허 번호 '1020170035116' 처리 중...
  데이터 생성 중...
특허 번호 '1020170035116' 처리 완료 (총 5개 데이터 생성)
특허 번호 '1020170035118' 처리 중...
  데이터 생성 중...
특허 번호 '1020170035118' 처리 완료 (총 5개 데이터 생성)
특허 번호 '1020170035122' 처리 중...
  데이터 생성 중...
특허 번호 '1020170035122' 처리 완료 (총 5개 데이터 생성)
특허 번호 '1020170035125' 처리 중...
  데이터 생성 중...
특허 번호 '1020170035125' 처리 완료 (총 5개 데이터 생성)
특허 번호 '1020170035126' 처리 중...
  데이터 생성 중...
특허 번호 '1020170035126' 처리 완료 (총 5개 데이터 생성)
특허 번호 '1020170035127' 처리 중...
  데이터 생성 중...
특허 번호 '1020170035127' 처리 완료 (총 5개 데이터 생성)
특허 번호 '1020170035129' 처리 중...
  데이터 생성 중...
특허 번호 '1020170035129' 처리 완료 (총 5개 데이터 생성)
특허 번호 '1020170035130' 처리 중...
  데이터 생성 중...
특허 번호 '1020170035130' 처리 완료 (총 5개 데이터 생성)
특허 번호 '1020170035133' 처리 중...
  데이터 생성 중...
특허 번호 '1020170035133' 처리 완료 (총 5개 데이터 생성)
특허 번호 '1020170035136' 처리 중...
  데이터 생성 중...
특허 번호 '102